In [1]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
from keras.layers import Input
import numpy as np
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from keras.optimizers import Adam, Nadam
from callbacks import all_callbacks
import pandas as pd
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import yaml
from train import parse_config, get_features

Using TensorFlow backend.


In [2]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputFile tree config outputDir")

options = Option(
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_conv2d.yml',
    outputDir = '../conv2d_weights/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_conv2d.yml
{'Inputs': ['j1_etarot', 'j1_phirot', 'j1_ptrel', 'j_index'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index'], 'KerasModel': 'conv2d_model', 'KerasModelRetrain': 'conv2d_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0.0001, 'NormalizeInputs': 0, 'InputType': 'Conv2D', 'BinsX': 40, 'MinX': -0.8, 'MaxX': 0.8, 'BinsY': 40, 'MinY': -1.0, 'MaxY': 1.0, 'MaxParticles': 100}


In [3]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

Keys: <KeysViewHDF5 ['t_allpar_new']>
load 24816
load 0
load 1000
load 2000
load 3000
load 4000
load 5000
load 6000
load 7000
load 8000
load 9000
load 10000
load 11000
load 12000
load 13000
load 14000
load 15000
load 16000
load 17000
load 18000
load 19000
load 20000
load 21000
load 22000
load 23000
load 24000
(24816, 40, 40, 1)
(19852, 40, 40, 1)
(4964, 40, 40, 1)


In [5]:
from models.models import conv2d_model
keras_model = conv2d_model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])


callbacks=all_callbacks(stop_patience=1000, 
                        lr_factor=0.5,
                        lr_patience=10,
                        lr_epsilon=0.000001, 
                        lr_cooldown=2, 
                        lr_minimum=0.0000001,
                        outputDir=options.outputDir)

keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 1)         0         
_________________________________________________________________
conv1_relu (Conv2D)          (None, 40, 40, 8)         976       
_________________________________________________________________
conv2_relu (Conv2D)          (None, 20, 20, 4)         292       
_________________________________________________________________
conv3_relu (Conv2D)          (None, 10, 10, 2)         74        
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                6432      
_________________________________________________________________
output_softmax (Dense)       (None, 5)                 165       
Total 


Epoch 00006: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 7/100
14889/14889 [==============================] - 80s 5ms/step - loss: 1.3982 - acc: 0.8028 - val_loss: 1.3534 - val_acc: 0.8072

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00007: val_loss improved from 1.43142 to 1.35344, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00007: val_loss improved from 1.43142 to 1.35344, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00007: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00007: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 8/100
14889/14889 [==============================] - 80s 5ms/step - loss: 1.3127 - acc: 0.8028 - val_loss: 1.2582 - val_acc: 0.8072

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00008: val_loss improved from 1.35344 to 1.25820, saving model

Epoch 00019: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 20/100
14889/14889 [==============================] - 78s 5ms/step - loss: 0.4090 - acc: 0.8080 - val_loss: 0.3871 - val_acc: 0.8271

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00020: val_loss improved from 0.41559 to 0.38709, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00020: val_loss improved from 0.41559 to 0.38709, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00020: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00020: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

Epoch 00020: saving model to ../conv2d_weights//KERAS_check_model_epoch20.h5

***callbacks end***

Epoch 21/100
14889/14889 [==============================] - 79s 5ms/step - loss: 0.3810 - acc: 0.8274 - val_loss: 0.3609 - val_acc: 0.8398

***callbacks***
saving losses to ../conv2d_weights/l

14889/14889 [==============================] - 58s 4ms/step - loss: 0.1755 - acc: 0.9700 - val_loss: 0.1713 - val_acc: 0.9698

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00033: val_loss improved from 0.18117 to 0.17135, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00033: val_loss improved from 0.18117 to 0.17135, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00033: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00033: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 34/100
14889/14889 [==============================] - 64s 4ms/step - loss: 0.1655 - acc: 0.9766 - val_loss: 0.1618 - val_acc: 0.9734

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00034: val_loss improved from 0.17135 to 0.16176, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00034: val_loss improved from 0.17135 to 0.16176, saving mod


Epoch 46/100
14889/14889 [==============================] - 68s 5ms/step - loss: 0.0788 - acc: 0.9994 - val_loss: 0.0793 - val_acc: 0.9992

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00046: val_loss improved from 0.08462 to 0.07933, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00046: val_loss improved from 0.08462 to 0.07933, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00046: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00046: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 47/100
14889/14889 [==============================] - 74s 5ms/step - loss: 0.0736 - acc: 0.9995 - val_loss: 0.0744 - val_acc: 0.9992

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00047: val_loss improved from 0.07933 to 0.07437, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00047: val_loss improved from 0.07933 to 0.074

14889/14889 [==============================] - 34s 2ms/step - loss: 0.0332 - acc: 0.9997 - val_loss: 0.0356 - val_acc: 0.9994

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00059: val_loss improved from 0.03776 to 0.03557, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00059: val_loss improved from 0.03776 to 0.03557, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00059: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00059: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 60/100
14889/14889 [==============================] - 29s 2ms/step - loss: 0.0311 - acc: 0.9997 - val_loss: 0.0336 - val_acc: 0.9994

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00060: val_loss improved from 0.03557 to 0.03364, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00060: val_loss improved from 0.03557 to 0.03364, saving mod

14889/14889 [==============================] - 26s 2ms/step - loss: 0.0093 - acc: 0.9997 - val_loss: 0.0131 - val_acc: 0.9996

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00085: val_loss improved from 0.01347 to 0.01314, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00085: val_loss improved from 0.01347 to 0.01314, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00085: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00085: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 86/100
14889/14889 [==============================] - 26s 2ms/step - loss: 0.0089 - acc: 0.9997 - val_loss: 0.0129 - val_acc: 0.9996

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00086: val_loss improved from 0.01314 to 0.01287, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00086: val_loss improved from 0.01314 to 0.01287, saving mod

14889/14889 [==============================] - 31s 2ms/step - loss: 0.0062 - acc: 0.9997 - val_loss: 0.0102 - val_acc: 0.9996

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00098: val_loss improved from 0.01033 to 0.01016, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00098: val_loss improved from 0.01033 to 0.01016, saving model to ../conv2d_weights//KERAS_check_best_model_weights.h5

Epoch 00098: saving model to ../conv2d_weights//KERAS_check_model_last.h5

Epoch 00098: saving model to ../conv2d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 99/100
14889/14889 [==============================] - 36s 2ms/step - loss: 0.0060 - acc: 0.9997 - val_loss: 0.0100 - val_acc: 0.9996

***callbacks***
saving losses to ../conv2d_weights/losses.log

Epoch 00099: val_loss improved from 0.01016 to 0.01002, saving model to ../conv2d_weights//KERAS_check_best_model.h5

Epoch 00099: val_loss improved from 0.01016 to 0.01002, saving mod

In [7]:
with open("../conv2d_weights/model.json", "w") as json_file:
    json_file.write(keras_model.to_json())